In [31]:
import pandas as pd
from samplics.estimation import TaylorEstimator

available = pd.read_csv("../auxiliary/ENOE_available.csv", names = ["path"]).path.to_list()
columns = pd.read_csv("../auxiliary/columns.csv").clave.to_list()
columns = [x.lower() for x in columns]

In [32]:
path = "data/ENOE/raw/2017t1/conjunto_de_datos_coe1_enoe_2017_1t/conjunto_de_datos/conjunto_de_datos_coe1_enoe_2017_1t.csv"

df = pd.read_csv('../'+path)

interseccion = list(set(df.columns) & set(columns))
df = pd.read_csv('../'+path, usecols=interseccion)
print(interseccion)

df.head()

['p2a_anio', 'ent', 'p4b', 'p3p1', 'p2_2', 'p3m8', 'upm', 'r_def', 'p1b', 'p3m2', 'p3q', 'p3m4', 'p3p2', 'p1a3', 'p3m7', 'p3o', 'p3r', 'fac', 'p2a_mes', 'p2_1', 'con', 'p3m6', 'p4a', 'p3m9', 'p1a1', 'h_mud', 'p1', 'p3m5', 'eda', 'p2_3', 'p1a2', 'cd_a', 'p3m1', 'p1c', 'p3m3']


,r_def,cd_a,ent,con,upm,h_mud,eda,p1,p1a1,p1a2,...,p3m8,p3m9,p3o,p3p1,p3p2,p3q,p3r,p4a,p4b,fac
0,0,83,7,50071,703409,0,37,1,,,...,,,2,,,5,1,8130,3,507
1,0,83,7,50071,703409,0,37,2,,,...,,,,,,,,,,507
2,0,83,7,50071,703409,0,14,2,,,...,,,,,,,,,,507
3,0,82,7,50023,702635,0,40,1,,,...,,,2,,,2,3,6112,2,510
4,0,82,7,50023,702635,0,38,1,,,...,,,,,,,3,7221,4,510


In [28]:
df.groupby(['p3o']).size()

p3o
     188132
1      2513
2    115735
dtype: int64

In [30]:
# WARNING: This takes a lot of time
prec = TaylorEstimator("total")
prec.estimate(
    y= df["p3o"],
    samp_weight=df['fac'],
    psu=df['upm']
)
print(prec)

In [ ]:
def results_estimation(estimator):
    text = str(estimator).replace(' ','|')

    # Transform to df
    prec = pd.DataFrame([x.split('|') for x in text.split('\n')])

    # Eliminar primera fila
    prec = prec.iloc[1:, 0:5]

    columnas = {prec.columns[0]: 'tot',
                prec.columns[1]: 'se',
                prec.columns[2]: 'low',
                prec.columns[3]: 'up',
                prec.columns[4]: 'cv',}

## Create dataset

We need a dataset with a structure such as

ent | year | quarter | em | unemp | mig | from_nem
| --| :---:|:---:|:---:|:---:|:---:|:---:|
dgo | 2017 | 1 | 0 | 0.23 | 0.12 | 0.1 |
dgo | 2017 | 2 | 0 | 0.23 | 0.12 | 0.1 |
dgo | 2017 | 3 | 0 | 0.23 | 0.12 | 0.1 |
dgo | 2017 | 4 | 0 | 0.23 | 0.12 | 0.1 |
dgo | 2018 | 1 | 0 | 0.23 | 0.12 | 0.1 |
dgo | 2018 | 2 | 0 | 0.23 | 0.12 | 0.1 |
dgo | 2018 | 3 | 0 | 0.23 | 0.12 | 0.1 |
dgo | 2018 | 4 | 0 | 0.23 | 0.12 | 0.1 |
cdmx | 2017 | 1 | 0 | 0.23 | 0.12 | 0.1 |
cdmx | 2017 | 2 | 0 | 0.23 | 0.12 | 0.1 |
cdmx | 2017 | 3 | 0 | 0.23 | 0.12 | 0.1 |
cdmx | 2017 | 4 | 0 | 0.23 | 0.12 | 0.1 |


In [6]:
import pandas as pd


migrated_from_path = "../data/ENOE/raw/2017t1/conjunto_de_datos_coe1_enoe_2017_1t/catalogos/p3p2.csv"
migrated_from = pd.read_csv(migrated_from_path, encoding='latin-1')
migrated_from.to_csv("../auxiliary/destinations_mig.csv")

In [33]:
# Lets only use people at work!
print(df.shape)
df = df[df['p3o'] != " "]
print(df.shape)
df.groupby(['ent','p3o']).size()

(306380, 35)
(118248, 35)


ent  p3o
1    1        26
     2      3830
2    1        30
     2      4078
3    1       216
            ... 
30   2      3211
31   1        74
     2      3447
32   1       149
     2      3403
Length: 64, dtype: int64

In [34]:
df['p3o'] = 2- pd.to_numeric(df['p3o'])
df['p3o'].mean()

0.02125194506460997

In [37]:
df.groupby(['ent']).p3o.mean()

ent
1     0.006743
2     0.007303
3     0.069880
4     0.021238
5     0.028805
6     0.029547
7     0.009675
8     0.008239
9     0.014247
10    0.027043
11    0.007469
12    0.016269
13    0.016361
14    0.007885
15    0.003564
16    0.012720
17    0.006321
18    0.028718
19    0.027778
20    0.038336
21    0.010698
22    0.014321
23    0.052571
24    0.020239
25    0.017744
26    0.036368
27    0.041740
28    0.037254
29    0.011923
30    0.015635
31    0.021017
32    0.041948
Name: p3o, dtype: float64